In [53]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Thông tin kết nối PostgreSQL
dbname = "postgres"
user = "postgres"
password = "login1234"
host = "localhost"
port = "5432"

In [45]:
# Kết nối đến PostgreSQL
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Cursor để thực thi các lệnh SQL
cursor = conn.cursor()

# Tên bảng trong PostgreSQL
table_name = "dwh.account"

# cursor.execute("select * from dwh.target")
# rows=cursor.fetchall()
# print(rows)
# Lặp qua từng dòng trong DataFrame và chèn vào bảng

query = "SELECT account_number FROM dwh.account"
cursor.execute(query)

account = []
for i in cursor.fetchall():
    account.append(i[0])

print(account[0])

744761900043


In [50]:
# Số lượng account_number
num_accounts = len(account)

# Sinh ngẫu nhiên số dư balance từ 0 đến 1,000,000
balances = np.random.randint(0, 1000001, num_accounts)

# Tạo DataFrame balance từ df_account và balances
balance = pd.DataFrame({
    'account_number': account,
    'balance': balances,
    'data_datetime': '2024-01-01'
})

# In ra một vài dòng đầu của balance để kiểm tra
# print(type(balance))

In [51]:
balance

,account_number,balance,data_datetime
0,744761900043,910291,2024-01-01
1,555286461200,536893,2024-01-01
2,539668218911,864809,2024-01-01
3,482534465189,776282,2024-01-01
4,119790767773,61400,2024-01-01
...,...,...,...
7995,607420946942,591465,2024-01-01
7996,421138292399,200845,2024-01-01
7997,276337860541,123185,2024-01-01
7998,911998841369,704075,2024-01-01


In [54]:

# Lặp qua từng dòng trong DataFrame và chèn vào bảng
for index, row in balance.iterrows():
    insert_query = sql.SQL("INSERT INTO dwh.balance ({}) VALUES ({})").format(
        # sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, balance.columns)),
        sql.SQL(', ').join(map(sql.Literal, row))
    )
    cursor.execute(insert_query)

# Lưu thay đổi và đóng kết nối
conn.commit()
cursor.close()
conn.close()